# Multi-effect distillation (MED) model validation

TODO: Use pre-processing and data conditioning functions from `models_psa.utils`


In [3]:
import os
%load_ext autoreload

# os.environ["MR"] = f"{os.environ['HOME']}/PSA/MATLAB_runtime/R2023b"
os.environ["MR"] = f"{os.environ['HOME']}/MATLAB/R2023b"
MR = os.environ["MR"]
os.environ["LD_LIBRARY_PATH"] = f"{MR}/runtime/glnxa64:{MR}/bin/glnxa64:{MR}/sys/os/glnxa64:{MR}/sys/opengl/lib/glnxa64"

from pathlib import Path
import os
import hjson
import json
import numpy as np
import pandas as pd
import time
from loguru import logger
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot

import MED_model
import matlab
import re

import sys

sys.path.append('..')

from solarMED_modeling.validation import within_range_or_nan_or_max, within_range_or_zero_or_max

resample_figures = False

# Visualize result so far
with open( Path("../data") / 'variables_config.hjson') as f:
    vars_config = hjson.load(f)
    
df = pd.read_csv('../data/dev.csv', parse_dates=['time'], index_col='time')

MED_model_matlab = MED_model.initialize()
logger.info('MATLAB engine initialized')


In [1]:
plt_config = {
  # General plot attributes
  "title": "MED",
  "subtitle": "Model validation",
  "height": 1200,
  "width": 1000,
  "margin": {
    "l":20,
    "r":100,
    "t":100,
    "b":20,
    "pad":5
  },
  "vertical_spacing": 0.03,
  "xdomain": [0, 0.85],
  "arrow_xrel_pos": 60, # seconds

  "plots": {
    "med_flows": {
      "title": "<b>MED</b>",
      "row_height": 1,
      "bg_color": "bg_gray",
      "ylabels_left": ["m<sup>3</sup>/h"],
      "ylabels_right": ["m<sup>3</sup>/h"],
      "tigth_vertical_spacing": True,
      "ylims_left": 'manual',

      # Plotted between this plot and the next one
      "show_active": True,
      "active_var_id": "med_active",
      "active_color": "dc_green_rgb", # Needs to end with _rgb

      "traces_left": [
        {
          "var_id": "qmed_f",
          "color": "c_blue",
          "mode": "lines",
          "dash": "dot"
        },
        {
          "var_id": "qmed_d",
          "color": "plotly_green",
          "mode": "lines",
          "dash": "dashdot"
        },
        {
          "var_id": "qmed_c",
          "color": "plotly_cyan",
          "mode": "lines",
          "dash": "dash",
        }
      ],

      "traces_right": [
        {
          "var_id": "qmed_s",
          "mode": "lines",
          "color": "plotly_red",
          "axis_arrow": True,
          "arrow_yrel_pos": 0.9,
        },
      ]
    },

    "med_temperatures": {
      "row_height": 1,
      "bg_color": "bg_gray", # bg gray
      "ylabels_left": ["⁰C"],
      "ylims_left": 'manual',

      "traces_left": [
        {
          "var_id": "Tmed_s_in",
          "mode": "lines",
          "color": "plotly_red",
        },
        {
          "var_id": "Tmed_s_out",
          "mode": "lines",
          "color": "plotly_red",
          "dash": "dash",
        },
        {
          "var_id": "Tmed_c_in",
          "mode": "lines",
          "color": "c_blue",
        },
        {
          "var_id": "Tmed_c_out",
          "mode": "lines",
          "color": "c_blue",
          "dash": "dash"
        },
      ]
    },

  }

}

In [9]:
# # Save to json
# import json
# 
# with open( Path("../data") / 'plt_config_med.json', 'w') as f:
#     json.dump(plt_config, f, indent=4)

In [48]:
idx_start = 500
span = 70
idx_end = len(df)
df_mod = pd.DataFrame()

# Run model
for idx in range(idx_start,idx_end):
    ds = df.iloc[idx]
        
    logger.info(f"Iteration {idx} / {len(df)}")
    start_time = time.time()

    Tmed_c_out = ds["Tmed_c_out"]
    Tmed_c_in = ds["Tmed_c_in"]
    mmed_f = ds["qmed_f"] if abs(ds["qmed_f"]-5) > 0.5 else 5
    mmed_f = within_range_or_nan_or_max(mmed_f, range=(5, 9), var_name="qmed_f")
    mmed_s = within_range_or_zero_or_max(ds["qmed_s"], range=(30, 48), var_name="qmed_s")
    Tmed_s_in = within_range_or_nan_or_max(ds["Tmed_s_in"], range=(60, 75), var_name="Tmed_s_in")

    MsIn = matlab.double([mmed_s / 3.6], size=(1, 1))  # m³/h -> L/s
    TsinIn = matlab.double([Tmed_s_in], size=(1, 1))
    MfIn = matlab.double([mmed_f], size=(1, 1))
    TcwinIn = matlab.double([Tmed_c_in], size=(1, 1))
    op_timeIn = matlab.double([0], size=(1, 1))
                
    med_model_solved = False
    mmed_d = np.nan
    Tmed_s_out = np.nan
    mmed_c = np.nan
    while not med_model_solved and (Tmed_c_in < Tmed_c_out < 40) and mmed_f > 0 and mmed_s > 0 and Tmed_s_in > 0 and Tmed_s_in > 0:   
        # try:                     
        TcwoutIn = matlab.double([Tmed_c_out], size=(1, 1))
    
        mmed_d, Tmed_s_out, mmed_c, _, _ = MED_model_matlab.MED_model(
            MsIn,  # L/s
            TsinIn,  # ºC
            MfIn,  # m³/h
            TcwoutIn,  # ºC
            TcwinIn,  # ºC
            op_timeIn,  # hours
            nargout=5
        )

        if mmed_c > 20:
            Tmed_c_out = Tmed_c_out + 1
        elif mmed_c < 9:
            Tmed_c_out = Tmed_c_out - 1
        else:
            med_model_solved = True
            
        # except Exception as e:
        #     # TODO: Put the right variable in the search and set the delta to the right value
        #     if re.search('mmed_c', str(e)):
        #         # self.penalty = self.default_penalty
        #         deltaTmed_cout = 0.5  # or -0.5
        #         logger.warning(f"Unfeasible operation in MED: {e}")
        #     else:
        #         raise e
        # 
        # # La dirección de cambio debería ser en función si el caudal de refrigeración es poco o demasiado
        # Tmed_c_out = np.min([Tmed_c_out + 0.5, 40])
        # Tmed_c_out = np.max([Tmed_c_out, ds["Tmed_c_in"]])
        # 
        # TcwoutIn = matlab.double([Tmed_c_out], size=(1, 1))
                  
    logger.info(f"Elapsed time: {time.time()-start_time:.2f} seconds")
    
    result = pd.DataFrame(
        {
            "qmed_f": mmed_f,
            "qmed_d": mmed_d,
            "qmed_c": mmed_c,
            "qmed_s": mmed_s,
            "Tmed_s_in": Tmed_s_in,
            "Tmed_s_out": Tmed_s_out,
            "Tmed_c_in": ds["Tmed_c_in"],
            "Tmed_c_out": Tmed_c_out,
        },
        index=[0]
    )
    
    df_mod = pd.concat([df_mod, result], ignore_index=True)

In [50]:
# Sync model index with measured data
df_mod.index = df.index[idx_start:idx if idx < idx_end - 1 else idx_end]

%autoreload 2

fig = experimental_results_plot(plt_config, df, df_comp=df_mod, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'med_validation_{df.index[0].strftime("%Y%m%d")}')
)